# Assessment

Congratulations on finishing the MPL course!! Hopefully, you learned some valuable skills along the way and had fun doing it. Now it is time to put those skills to the test. In this assessment, you will train a new model capable of classifying the 10 different classes all included in the database known as [_CIFAR-10_](https://www.cs.toronto.edu/~kriz/cifar.html). You will need to get the model to a validation to pass the assessment, although we challenge you to do even better if you can. You will have to use the skills you learned in the previous exercises. Let's get started! 

## The Problem: CIFAR-10 Classification

<p style='text-align: justify;'>
To start studying neural networks, it is necessary to understand and be able to apply the most basic concepts of recognition to your model. A good way to test and train your model is to use the CIFAR-10 database which consists of a set of $60,000$ images for $10$ different classes (planes, cars, birds, cats, deer, dogs, frogs, horses, ships and trucks) which are identified from $0$ to $9$ respectively. With that in mind, you will train your model on top of this base, making it possible to correctly recognize each element of these classes.
</p>    

<p style="text-align: center;">
 <img src="../images/cifar-10.png">
</p>

### Characteristics of images from the CIFAR-10 database

The images have dimensions of $32$x$32$ so remember to adjust your image when making a prediction with the model, for each class there are $6,000$ images related to its type so you must best adjust the number of training epochs as well as the learning rate.

Then:

1. Create an MLP Neural Network.

2. Import the CIFAR-10 database from the torchvision library.

3. Train your model so that it correctly performs predictions, paying attention to the input parameters.

4. Perform the prediction with some image.

### ☆ Solution ☆ 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import os
from PIL import Image


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

input_size = 3072  # input size (32 x 32)
hidden_size = 128
output_size = 10
learning_rate = 0.01
num_epochs = 100

# Convert to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load torchvision CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

model = MLP(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


# Loop
for epoch in range(num_epochs):
    total_loss = 0
    for images, labels in trainloader:
        # Resize images to a one-dimensional vector
        images = images.view(-1, input_size)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimzation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss each epoch
    average_loss = total_loss / len(trainloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

image = Image.open('./images/validation-model-cifar10-cat.jpg') # Enter the path to your image here

# Image pre-processing
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to expected input size
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])
image = transform(image)

# Add an extra dimension in the image tensor and resize image
image = image.unsqueeze(0)
image = image.view(-1, input_size)

# Passing through the neural network
model.eval()  # Change model mode to evaluation
with torch.no_grad():
    output = model(image)

# Results
probabilities = torch.softmax(output, dim=1)
predicted_class = torch.argmax(probabilities, dim=1).item()

classes_Cifar10 = ["Airplane","Car","Bird","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]

print(f"Expected class: {classes_Cifar10[predicted_class]}")

## Clear the Memory

Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
#import IPython
#app = IPython.Application.instance()
#app.kernel.do_shutdown(True)

## Clear the Temporary Files

Before finished the assessment, please execute the following cell to clear up the directory.

In [1]:
!rm -rf ../models/handwritten-model.pt ../datasets/cifar-10-python.tar.gz  ../datasets/cifar-10-batches-py ../datasets/cifar-10-batches-py ../datasets/MNIST  cifar-10*  